* Author: HZQ
* Last modified: 2018/12/4

# Main Parts

## prepared process

### Import required packages

### load data offline

### load data online

In [ ]:
import logging

In [ ]:
class process_pre_c():
    
    def __init__(self):
        allocate needed folder path and path checking
        allocate target urls
        allocate strategy for error catch, anti-spider breaking
    def reduce_mem(self):
        reduce memory use by changing data type of each dataframe or other techs 
    def load_online(self):   
        def init_spider():
            choose creator from scrapy, request+BS and so on 
    
        def generate_spider():
            application of spider by choosed method
    
        def excute_spider():
            excute spider object and return structual texts
            save dataframe object named like xxx_ori
    
        def save_data():
            save dataframe to ".../data_raw"
    def load_offline(self):
        def read_data(big_flag):
            if big_flag:
                use chunksize params in read_csv
            else:
                read normally

## middle process

In [ ]:
class process_mid_c():
    def __init__(self):
        allocate room for the variable returned by class process_pre_c
        add variables required by the below functions

    def apply_checkings(self):
        application of serval checkings to the dataframe from statistical checking, completion, repetability, survivor bias
    
    def mark_correlation(self):
        from description file extract feat descriptions of each data file
        use graph structure or database structure to represent relationships between each dataset
        save correlation parameters between each dataset to csv files
    def divide_datatype(self):
        divide data in serveral types including categorical data(nit include binary data), mixed data(need further extraction), binary data(no normalization needed), numerical data and so on
        
    def expand_data(self):
        deal with outliers, record reasons annotated with date
        deal with mixed data, record reasons annotated with date
        deal with feats from other csv files or construct new feat, record reasons annotated with date
        
    def save_data(self):
        save dataframe to ".../data_processed"

### check data properties

#### check statistical characters

#### check completion

#### check survivor bias

#### check time-related

### divide data category

## last process

In [ ]:
class process_last(self):   
    def __init__(self):
        allocate room for the variable returned by class process_mid_c
        add variables required by the below functions
        
    def fill_na(self):
        use Imputer() object from sklearn or the attribute of dataframe ".fillna(method=None, mode=None, inplace=False/True)"
        save dataframe to ".../data_processed"  
    
    def apply_scaler(self):
        use different scalers provided by sklearn with consideration of model feed
        save dataframe to ".../data_processed"
        
    def select_feat(self):
        apply algorithms like RF(SL), Kmeans(UL) or PCA on data
        save dataframe to ".../data_processed"
    


# Test Parts